In [4]:
import tensorrt
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
import torch
print(torch.rand(2, 2))

tensor([[0.0585, 0.7011],
        [0.2547, 0.9405]])


In [ ]:
import sys
sys.path.append('/g/g11/quyng/Documents/code/Cardiac-electrophysiology/T3_ansh')
import data_dssi as ds # name clash, will import from current folder instead!
import utils

import torch
from torch.utils.data import DataLoader, random_split
import datetime
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
split_size = [0.8, 0.15, 0.05]
batch_size = 2 #64

data = ds.read_data()
ds_data = ds.Custom_dataset(data)

generator = torch.Generator().manual_seed(42)
train_ds, val_ds, test_ds = random_split(ds_data, split_size, generator=generator)

# Create data loaders for our datasets; shuffle for training, not for validation
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False)

# Checkpoint
print('Train:')
for batch in train_loader:
    print('Input dimensions:', batch[0].shape)
    print('Output dimensions:', batch[1].shape)
    
print('Validation:')
for batch in val_loader:
    print('Input dimensions:', batch[0].shape)
    print('Output dimensions:', batch[1].shape)

print('Test:')    
for batch in test_loader:
    print('Input dimensions:', batch[0].shape)
    print('Output dimensions:', batch[1].shape)

In [ ]:
import math, os, time
from tempfile import TemporaryDirectory
from typing import Tuple

from torch import nn, Tensor
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import dataset

In [ ]:
class SqueezeNet(nn.Module):

    def __init__(self,
        input_height: int=500,
        input_len: int=12,
        batch_first: bool=True,  
        output_len: int=75, 
        num_hidden: int=512):
        
        super().__init__()
        
        self.model_type = '1D-CNN'
        
        self.squeeze = nn.Sequential(
            # Convert from 2D to 1D
            nn.Conv2d(1, 32, kernel_size=(3, input_len), stride=1, padding=(1, 0)),
            nn.ReLU()
        )

        self.conv = nn.Sequential(
            # Convolutional block
            nn.Conv1d(32, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv1d(128, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv1d(32, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv1d(256, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv1d(64, 512, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv1d(512, 25, kernel_size=3, stride=1, padding=1),
            nn.ReLU()
        )

        
    def forward(self, src: Tensor) -> Tensor:
        
        # input is torch.Size([2, 500, 12])
        output = self.squeeze(src.unsqueeze(1)) # torch.Size([2, 16, 500, 1])
        output = output.squeeze(3) # torch.Size([2, 16, 500])
        
        output1 = self.conv(output) # torch.Size([2, 25, 500])
        output2 = self.conv(output) # torch.Size([2, 25, 500])
        output3 = self.conv(output) # torch.Size([2, 25, 500])

        final = torch.cat([output1, output2, output3], 1) # should be torch.Size([2, 75, 500])
        
        return final


def train(model: nn.Module) -> None:
    
    model.train()  # turn on train mode
    
    total_loss = 0.
    print('epochs : ', epoch)

    for x, y in train_loader:

        data, targets = x.to(device), y.to(device)
        
        output = model(data)

        # need to fix
        targets = targets.reshape(-1, 75)
        loss = criterion(output, targets)

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
    
    return total_loss/batch_size


def evaluate(model: nn.Module, eval_data: Tensor) -> float:
    
    model.eval()  # turn on evaluation mode
    
    total_loss = 0.
    batch_l = len(eval_data) # dim 0 = batch size
    
    with torch.no_grad():
        for x, y in eval_data:
            
            data, targets = x.to(device), y.to(device)

            output = model(data)

            # need to fix
            targets = targets.reshape(-1, 75)
            total_loss += criterion(output, targets).item()

    return total_loss/batch_l


def plotActTime(output: np.array) -> None:

    plt.figure(figsize=(1, 10))

    output = output.view(75,-1)

    ActTime = output.detach().numpy()

    # plot the Activation Time array
    plt.imshow(ActTime, cmap='jet', interpolation='nearest', aspect='auto')
    plt.title('Activation Time')
    plt.colorbar()
    plt.grid(visible=True, which='major', color='#666666', linestyle='-')
    plt.minorticks_on()
    # not xticks
    plt.xticks([])
    plt.grid(visible=True, which='minor', color='#999999', linestyle='-', alpha=0.2)
    plt.show()



In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

epochs = 3

model = SqueezeNet().to(device)

criterion = torch.nn.MSELoss()
lr = 5.0  # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

best_val_loss = float('inf')
t_loss = []
v_loss = []

with TemporaryDirectory() as tempdir:

    best_model_params_path = os.path.join(tempdir, "best_model_params.pt")

    for epoch in range(1, epochs + 1):
        epoch_start_time = time.time()
        #print('trainer is running wild')

        tr_loss = train(model)
        val_loss = evaluate(model, val_loader)

        t_loss.append(tr_loss)
        v_loss.append(val_loss)

        elapsed = time.time() - epoch_start_time
        print('-' * 89)
        print(f'| end of epoch {epoch:3d} | time: {elapsed:5.2f}s | '
            f'valid loss {val_loss:5.2f} | ' f'train loss {tr_loss:5.2f}' )
        print('-' * 89)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), best_model_params_path)

        scheduler.step()
    
    fig, axes = plt.subplots(1, 2, figsize=(5, 10))
    ax = axes[0]
    ax.plot(range(len(t_loss)), t_loss)
    ax.set_ylabel('Training loss')
    ax.set_xlabel('Epoch')
    ax = axes[1]
    ax.plot(range(len(v_loss)), v_loss)
    ax.set_ylabel('Validation loss')
    ax.set_xlabel('Epoch')

    model.load_state_dict(torch.load(best_model_params_path)) # load best model states

    test_loss = evaluate(model, test_loader)

    print('=' * 89)
    print(f'| End of training | test loss {test_loss:5.2f}')
    print('=' * 89)